In [1]:
import os
import numpy as np
import flask
import pickle
from flask import Flask, render_template, request, jsonify
import json
import os
import tarfile
import warnings
import numpy as np
import pandas as pd
import requests
import spacy
# import wget
from spacy.lang.en import English
import scattertext as st
from json import loads
from lxml import html
from requests import Session
from concurrent.futures import ThreadPoolExecutor as Executor
from itertools import count
from flask import jsonify
import requests
# from flask_cors import CORS
warnings.filterwarnings('ignore')


#new
from spacy.tokenizer import Tokenizer

# nlp = spacy.load("./down_sm/en_core_web_sm-2.1.0/en_core_web_sm/en_core_web_sm-2.1.0")
nlp = spacy.load("en_core_web_sm")
# yelp_url = 'https://www.yelp.com/biz/nomadworks-new-york-2?osq=nomadworks'
base_url = "https://www.yelp.com/biz/" # add business id
api_url = "/review_feed?sort_by=date_desc&start="
bid = "amélie-new-york" # business id

class Scraper():
    def __init__(self):
        self.data = pd.DataFrame()

    def get_data(self, n, bid=bid):
        with Session() as s:
            with s.get(base_url+bid+api_url+str(n*20)) as resp: #makes an http get request to given url and returns response as json
                r = loads(resp.content) #converts json response into a dictionary
                _html = html.fromstring(r['review_list']) #loads from dictionary

                dates = _html.xpath("//div[@class='review-content']/descendant::span[@class='rating-qualifier']/text()")
                reviews = [el.text for el in _html.xpath("//div[@class='review-content']/p")]
                ratings = _html.xpath("//div[@class='review-content']/descendant::div[@class='biz-rating__stars']/div/@title")

                df = pd.DataFrame([dates, reviews, ratings]).T

                self.data = pd.concat([self.data,df])

    def scrape(self): #makes it faster
        # multithreaded looping
        with Executor(max_workers=40) as e:
            list(e.map(self.get_data, range(10)))


s = Scraper()
s.scrape()
df = s.data
df.columns = ['date','review','rating']
df.head(2)

ModuleNotFoundError: No module named 'spacy'

In [ ]:
import sys
! conda install -c conda-forge spacy

In [388]:
pd.options.display.max_colwidth = 2000
df = df.set_index(df.columns.drop('review',1).tolist()).review.str.split('.', expand=True).stack().reset_index().rename(columns={0:'review'}).loc[:, df.columns]

df = df.replace(',','')
df = df.replace('!','')
df = df.replace('#','')
df = df.replace('.','')
df.head(2)

,date,review,rating
0,\n 5/2/2019\n,"This review is a year late, but I need to mark this down as a very memorable moment",5.0 star rating
1,\n 5/2/2019\n,,5.0 star rating


In [389]:
# df['review'] = df['review'].str.cat(sep='0')


In [390]:
tokenizer = Tokenizer(nlp.vocab)
STOP_WORDS = nlp.Defaults.stop_words.union(['gets','incredible','disappoint','from','perfection','loved','definitely','happy','find','found','simply','fantastic','recommend','feel','little','i','wow','absolute','favorite','excellent','delicious','great','maybe','very','enjoy','list','gave','date','went','disappointed','nyc','got','#','crazy','other','fairness','fair','mid','from','highly','perfect','perfectly','come','lovely','visit','ny','nyc','best','amazing','love','absolutely','like','good','other','from','ny','restaurant','we','will','because','not','friends','amazing','awesome','first','he','check-in','=','= =','male','u','want', 'u want', 'cuz','him',"i've", 'deaf','on', 'her','told','told him','ins', 'check-ins','check-in','check','I', 'i"m', 'i', 'it', "it's", 'it.','they','coffee','place','they', 'the', 'this','its', 'l','-','they','this','don"t','the ',' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','(',')','/','.',',','!'])
# STOP_WORDS
df = df[df['review'] !=None]
# df['review'] = [str(x) for x in df['review']]
# df['review'] = df['review'].str.cat(sep='0')
# df['review'][0]

In [391]:
# nlp.Defaults.stop_words |= {'will','because','not','friends','amazing','awesome','first','he','check-in','=','= =','male','u','want', 'u want', 'cuz','him',"i've", 'deaf','on', 'her','told','told him','ins', 'check-ins','check-in','check','I', 'i"m', 'i', ' ', 'it', "it's", 'it.','they','coffee','place','they', 'the', 'this','its', 'l','-','they','this','don"t','the ', ' the', 'it', 'i"ve', 'i"m', '!', '1','2','3','4', '5','6','7','8','9','0','/','.',','}

tokens = []

for doc in tokenizer.pipe(df['review'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc: 
        if (token.text not in STOP_WORDS) & (token.is_punct == False):
            doc_tokens.append(token.text.lower())
    tokens.append(doc_tokens)
    
df['review'] = tokens

In [392]:
jointty = lambda x: ' '.join(map(lambda x: str(x), x['review']))
df['review'] = df.apply(jointty, axis=1)
df.head(1)

,date,review,rating
0,\n 5/2/2019\n,"this review year late, need mark memorable moment",5.0 star rating


In [406]:
df['review'].replace(' ', np.nan, inplace=True)
df= df.dropna()
df.head(5)

,date,review,rating
0,\n 5/2/2019\n,"this review year late, need mark memorable moment",5.0 star rating
2,\n 5/1/2019\n,"amelie far new restaurant! the food incredible, wine divine! #yum",5.0 star rating
3,\n 4/30/2019\n,got wine flight,4.0 star rating
4,\n 4/30/2019\n,the menu said comes waiter said hour clearly hour time selections,4.0 star rating
6,\n 4/29/2019\n,tapas,4.0 star rating


In [394]:
corpus = (st.CorpusFromPandas(df,
                         category_col='rating',
                         text_col='review',
                         nlp=nlp).build().remove_terms(STOP_WORDS, ignore_absences=True))
term_freq_df = corpus.get_term_freq_df()
term_freq_df['highratingscore'] = corpus.get_scaled_f_scores('5.0 star rating')

term_freq_df['poorratingscore'] = corpus.get_scaled_f_scores('1.0 star rating')

In [395]:
dp = term_freq_df.sort_values(by= 'poorratingscore', ascending = False)
# listofchshouldnotcontain = [' ', '!',',','#','(',')']
dp = dp[~dp.index.str.contains('-')]
dp = dp[~dp.index.str.contains("'")]
dp = dp[~dp.index.str.contains('/')]
dp.head(30)

,5.0 star rating freq,4.0 star rating freq,3.0 star rating freq,2.0 star rating freq,1.0 star rating freq,highratingscore,poorratingscore
term,,,,,,,
octopus dish,0,0,0,0,2,0.207099,1.0
cockroach,0,0,0,0,2,0.207099,1.0
luke,0,0,0,0,1,0.440284,1.0
intimate decent,0,0,0,0,1,0.440284,1.0
service food,0,0,0,0,1,0.440284,1.0
flat,0,0,0,0,1,0.440284,1.0
frites octopus,0,0,0,0,1,0.440284,1.0
dish zucchini,0,0,0,0,1,0.440284,1.0
zucchini flat,0,0,0,0,1,0.440284,1.0


In [396]:
dh = term_freq_df.sort_values(by= 'highratingscore', ascending = False)
dh = dh[~dh.index.str.contains('-')]
dh = dh[~dh.index.str.contains("'")]
dh = dh[~dh.index.str.contains('/')]
dh.head(30)

,5.0 star rating freq,4.0 star rating freq,3.0 star rating freq,2.0 star rating freq,1.0 star rating freq,highratingscore,poorratingscore
term,,,,,,,
bottle,7,1,0,0,0,1.000000,1.528804e-02
amelie,18,2,1,0,0,0.995672,2.806343e-10
eggs,6,1,0,0,0,0.992788,3.148985e-02
food service,6,1,0,0,0,0.992788,3.148985e-02
glass,9,1,1,0,0,0.984267,1.033703e-03
birthday,4,0,0,0,0,0.980771,1.713667e-01
amélie,4,0,0,0,0,0.980771,1.713667e-01
love hour,4,0,0,0,0,0.980771,1.713667e-01
times,12,2,1,0,0,0.978525,7.759069e-06


In [397]:
dh.index[:99]

Index(['bottle', 'amelie', 'eggs', 'food service', 'glass', 'birthday',
       'amélie', 'love hour', 'times', 'chicken liver', 'liver', 'packed',
       'brunch', 'attentive', 'chicken', 'french food', 'charcuterie',
       'reservation', 'flight', 'new', 'mousse', 'fresh', 'hour wine',
       'servers', 'creme', 'tasting', 'grilled', 'service', 'escargot',
       'enjoyed', 'red', 'board', 'staff', 'the food', 'wonderful', 'friend',
       'experience', 'go to', 'visiting', 'charcuterie board', 'oh', 'husband',
       'tiny', 'notch', 'share', 'hours', 'great food', 'authentic', 'priced',
       'talking', 'true', 'quiet', 'live', 'ready', 'meats', 'wine food',
       'rushed', 'serve', 'wonderful experience', 'pure', 'great night',
       'weekend', 'the escargot', 'service great', 'ambience', 'gets', 'dish',
       'day', 'course', 'tasty', 'french', 'night', 'evening', 'trout',
       'frites', 'worth', 'early', 'tables', 'bar', 'cheese', 'duck', 'hour',
       'atmosphere', 'wine

In [398]:
dh.shape

(4271, 7)

In [399]:
dp.index[:99]

Index(['octopus dish', 'cockroach', 'luke', 'intimate decent', 'service food',
       'flat', 'frites octopus', 'dish zucchini', 'zucchini flat',
       'flat bread', 'cold', 'everything came', 'came cold', 'luke warm',
       'cockroach mid', 'mushy', 'texture', 'gross', 'dish mushy',
       'mushy texture', 'texture gross', 'subpar', 'know gets', 'gets high',
       'reviews subpar', 'subpar dishes', 'decent selection', 'other food',
       'atmosphere intimate', 'kill cockroach', 'fell from', 'sure i',
       'okay fairness', 'fairness wine', 'wine super', 'super deal',
       'deal food', 'food awesome', 'having kill', 'ceiling',
       'however cockroach', 'cockroach fell', 'fell', 'comp $', 'from the',
       'the ceiling', 'ceiling pay', 'comp', 'kill', 'flight having',
       'at comp', '12 wine', 'food atmosphere', 'high reviews', 'staff nice',
       'selection wine', 'zucchini', 'underwhelming', 'food underwhelming',
       'ordered steak', 'pay', 'wait staff', 'octopus', 'i